In [1]:
import sys
sys.path.append("..")
import lovely_tensors as lt
from tqdm import tqdm
import os
import pandas as pd
lt.monkey_patch()

In [2]:
from grins.data.mit_place_pulse.dataset import MITPlacePulseDataset
from grins.config import EXTERNAL_DATA_DIR

2025-09-21 12:45:03.224 | INFO     | grins.config:<module>:11 - PROJ_ROOT path is: /home/nicola/Development/GRINS


In [3]:
ds = MITPlacePulseDataset("../data/external/mit-place-pulse", split="train")

In [5]:
from torch.utils.data import DataLoader

dl = DataLoader(ds, batch_size=4, shuffle=True, collate_fn=ds.collate_fn)
for i, batch in enumerate(dl):
    print(batch)
    if i > 100:
        break

{'images0': tensor[4, 3, 300, 400] n=1440000 (5.5Mb) x∈[0., 1.000] μ=0.448 σ=0.246, 'images1': tensor[4, 3, 300, 400] n=1440000 (5.5Mb) x∈[0., 1.000] μ=0.421 σ=0.241, 'labels': tensor[4] i64 x∈[0, 1] μ=0.500 σ=0.577 [0, 1, 0, 1], 'image0_names': ['513d7b81fdc9f03587006bb9', '513d9f78fdc9f03587008750', '513d7bd9fdc9f03587006cd5', '50f607dcbeb2fed6f800033e'], 'image1_names': ['513929f4818ede1296001169', '5141404ffdc9f0492600609e', '513f2b94fdc9f0358700d70f', '513d7a13fdc9f035870066e7'], 'questions': ['livelier', 'safer', 'more beautiful', 'safer']}
{'images0': tensor[4, 3, 300, 400] n=1440000 (5.5Mb) x∈[0., 1.000] μ=0.417 σ=0.254, 'images1': tensor[4, 3, 300, 400] n=1440000 (5.5Mb) x∈[0., 1.000] μ=0.470 σ=0.220, 'labels': tensor[4] i64 x∈[0, 1] μ=0.500 σ=0.577 [1, 0, 0, 1], 'image0_names': ['513d7c2dfdc9f03587006df1', '513f2fc0fdc9f0358700de22', '50f55eb0fdc9f065f0004e1d', '513d9e66fdc9f035870083c9'], 'image1_names': ['513cc1eafdc9f03587001501', '513d9f99fdc9f03587008797', '50f445aefdc9f